In [57]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from multiprocessing.pool import Pool
import re
positive_file = 'rt-polarity.pos'
negative_file = 'rt-polarity.neg'
data_root = 'stanfordSentimentTreebank/'
glove_pattern = 'glove.6B.<size>d.txt'

In [60]:
def get_glove(size = 50):
    file = re.sub('<size>', str(size), glove_pattern)
    glove = pd.read_csv(file, sep = " ", header = None, index_col = 0, quoting = 3)
    glove = {key: val.values for key, val in glove.T.items()}
    return glove
glove = get_glove()

In [62]:
glove_words = set(glove.keys())

In [55]:
sentence_dict = {}
vocab = set([])
ttws = tf.keras.preprocessing.text.text_to_word_sequence
with open(data_root + 'datasetSentences.txt') as data:
    lines = data.readlines()
    first = True
    for line in lines:
        if first:
            first = False
            continue
        tokens = ttws(line)
        try:
            idx = tokens.pop(0)
            
        lim += 1
        if lim > 10:
            break

['sentence', 'index', 'sentence']
['1', 'the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'s", 'new', 'conan', "''", 'and', 'that', 'he', "'s", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', 'jean', 'claud', 'van', 'damme', 'or', 'steven', 'segal']
['2', 'the', 'gorgeously', 'elaborate', 'continuation', 'of', 'the', 'lord', 'of', 'the', 'rings', "''", 'trilogy', 'is', 'so', 'huge', 'that', 'a', 'column', 'of', 'words', 'can', 'not', 'adequately', 'describe', 'co', 'writer', 'director', 'peter', 'jackson', "'s", 'expanded', 'vision', 'of', 'j', 'r', 'r', 'tolkien', "'s", 'middle', 'earth']
['3', 'effective', 'but', 'too', 'tepid', 'biopic']
['4', 'if', 'you', 'sometimes', 'like', 'to', 'go', 'to', 'the', 'movies', 'to', 'have', 'fun', 'wasabi', 'is', 'a', 'good', 'place', 'to', 'start']
['5', 'emerges', 'as', 'something', 'rare', 'an', 'issue', 'movie', 'that', "'s", 'so', 'honest', 'and', 'keenly', 'observed', 'that', 

In [63]:
x = [0,1,1,23,3]
x.pop(0)

0

In [64]:
x

[1, 1, 23, 3]

In [52]:
model.wv['of']

array([ 0.13426188, -0.56361985,  0.32195488,  0.11725293,  0.3393347 ,
       -0.31241295,  0.44987684, -0.22493903,  0.36934644,  0.50523955,
        0.10486971, -0.33655983, -0.56142926,  0.34979793,  0.10301422,
        0.2026846 ,  0.36191243,  0.5586596 ,  0.8336534 ,  0.5481203 ,
        0.420142  ,  0.5527857 , -0.5410098 , -0.23286176,  0.7467346 ,
       -0.93865055,  0.55383164, -0.67054266,  0.4077901 ,  0.4391646 ,
        0.37475398,  0.5073329 , -0.25291252, -0.55292714, -0.07593767,
        0.32461494, -0.8306799 ,  0.08478513, -0.16767861, -0.6105417 ,
       -0.04402849,  0.62714684,  0.13303672,  0.2342712 , -0.5111181 ,
       -0.80780894, -0.9488481 ,  0.20399643,  0.40686616,  0.34480593,
       -0.20055702, -0.07647365,  0.11458617,  0.21996137, -0.34841064,
        0.6981789 ,  0.7910905 , -0.05794046, -0.49697298, -0.170719  ,
        0.7831816 ,  0.27283636,  0.23275974, -0.6248634 ,  0.52773106,
        0.6051587 , -0.05934991, -0.53924316,  0.0199111 ,  0.39